In [1]:
import codecs
import re
import numpy as np
import pandas as pd
import sys
import os
import glob
import time
path="./result_data/K"
place_catch="常滑"
data=pd.DataFrame(columns=["date","place","race_num","race_rank","position","goal","id","name","motor","boat","tenji","shinnyu","start","time","weather","wind_dir","wind_st","wave","kimarite"])

In [2]:
def data_clean(data):
    tmp=data.replace("\u3000","")
    tmp=tmp.replace("\r\n","")
    tmp=re.split(" +",tmp)
    return tmp

def get_weather(w1,w2):
    w1.pop(0)
    w2.pop(0)
    if w1[0].find("R")!=-1:
        return w1,w2[-1]
    if w2[0].find("R")!=-1:
        return w2,w1[-1]

In [3]:
def get_place(tmp):
    place=data_clean(tmp)
    return place


def add_data(date,test,each,weather,kimarite,place):
    st=re.split(" +",each)
    tmp_data=[st[2],st[1].replace("0",""),st[3],st[4].replace("\u3000",""),st[5],st[6],st[7],st[8],st[9],st[10].replace("\r\n","")]
    concat_data=[date,place]+[weather[0].replace("R","")]+[weather[1]]+tmp_data+[weather[-6],weather[-4]]+[weather[-3].replace("m","")]+[weather[-1].replace("cm","")]+[kimarite]

    add=pd.DataFrame([concat_data],columns=test.columns)
    #test.appned(pd.DataFrame([[st[1],st[2],st[4].replace("\u3000","")], columns=["rane","goal","name"]))
    #test=np.append(test,[st[1],st[2]],axis=0)
    #test=np.append(test,[add_data],axis=0)
    return add

In [4]:
def get_data(date,test,file,place_catch):
    line_num=0
    tmp=[-1,-1]
    with codecs.open(file,"r", "Shift-JIS", "ignore") as race:
        for i,each in enumerate(race):

            #ボートレース開催地を取得
            if each.find("ボートレース")!=-1:
                place_tmp=get_place(each)
                place=place_tmp[-1].replace("ボートレース","")

            #レース情報の前に発生するハイフン列を取得    
            if each.find("---")!=-1:
                tmp=[i+1,i+6]
                weather,kimarite=get_weather(line_odd,line_even)

            if line_num%2==0:
                line_odd=data_clean(each)
                #print(line_tmp_1)
                line_num+=1
            else:
                line_even=data_clean(each)
                #print("{}".format(line_tmp_2))
                line_num+=1

            # リストにデータを追加
            if i>=tmp[0] and i<=tmp[1] and place==place_catch:
                test=test.append(add_data(date,test,each,weather,kimarite,place_catch))
    return test

In [5]:
st=time.time()
for file in glob.glob(path+"*.TXT"):
    sys.stdout.write("\r call:{}".format(file))
    tmp_data=pd.DataFrame(columns=["date","place","race_num","race_rank","position","goal","id","name","motor","boat","tenji","shinnyu","start","time","weather","wind_dir","wind_st","wave","kimarite"])
    date=file.replace(path,"")
    date=date.replace(".TXT","")
    tmp_data=get_data(date,tmp_data,file,place_catch)
    data=pd.concat([data,tmp_data],axis=0)
ela=round(time.time()-st,2)
print("\nfinish:{}sec".format(ela))

 call:./result_data/K180227.TXT
finish:25.524890899658203sec


In [8]:
data.to_csv("csv_files/result.csv",encoding="shift_jis")